## Notebook for making predictions with deleted constant y samples


In [1]:
import pandas as pd
import sys
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor

### Reading dataset stored in other notebooks

In [2]:
sys.path.append('../')

%store -r dml

dm = dml

## Getting data

In [9]:
X_A_train = dm.train_a.iloc[:, 2:-1]
Y_A_train = dm.train_a.iloc[:, 0]

X_B_train = dm.train_b.iloc[:, 2:-1]
Y_B_train = dm.train_b.iloc[:, 0]

X_C_train = dm.train_c.iloc[:, 2:-1]
Y_C_train = dm.train_c.iloc[:, 0]

X_A_test = dm.X_test_estimated_a[dm.X_test_estimated_a.columns.intersection(X_A_train.columns)]
X_B_test = dm.X_test_estimated_b[dm.X_test_estimated_a.columns.intersection(X_A_train.columns)]
X_C_test = dm.X_test_estimated_c[dm.X_test_estimated_a.columns.intersection(X_A_train.columns)]

In [10]:
# Concat for common b and c model

X_BC_train = pd.concat([X_B_train, X_C_train])
Y_BC_train = pd.concat([Y_B_train, Y_C_train])

In [11]:
X_A_train.head(3)

,absolute_humidity_2m:gm3,ceiling_height_agl:m,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,fresh_snow_1h:cm,fresh_snow_24h:cm,is_in_shadow:idx,msl_pressure:hPa,precip_type_5min:idx,pressure_50m:hPa,prob_rime:p,rain_water:kgm2,relative_humidity_1000hPa:p,sfc_pressure:hPa,snow_melt_10min:mm,snow_water:kgm2,sun_elevation:d,total_cloud_cover:p,visibility:m,wind_speed_10m:ms,wind_speed_w_1000hPa:ms,month,season,hours,sum_rad:W
0,7.7,1744.900024,1744.900024,0.0,280.299988,0.0,0.0,1.0,1006.799988,0.0,1000.299988,0.0,0.0,73.099998,1006.299988,-0.0,0.1,-3.202,100.0,39640.101562,3.7,-0.0,0.878734,-0.913545,22,0.0
1,7.7,1734.000000,1734.000000,0.0,280.299988,0.0,0.0,1.0,1006.500000,0.0,999.900024,0.0,0.0,72.199997,1006.000000,-0.0,0.2,-3.650,100.0,40123.898438,3.6,-0.0,0.878734,-0.913545,22,0.0
2,7.7,1723.500000,1723.500000,0.0,280.299988,0.0,0.0,1.0,1006.099976,0.0,999.599976,0.0,0.0,71.199997,1005.599976,-0.0,0.2,-3.998,100.0,40628.300781,3.6,-0.0,0.878734,-0.913545,22,0.0


## Catboost models
will try with two models:
-   One for dataset A
-   One for B and C

In [12]:
catModel_a = CatBoostRegressor()
catModel_bc = CatBoostRegressor()

In [13]:
catModel_a.fit(X_A_train,Y_A_train)
catModel_bc.fit(X_BC_train,Y_BC_train)

Learning rate set to 0.088994
0:	learn: 1073.3390987	total: 9.72ms	remaining: 9.71s
1:	learn: 1001.3279862	total: 17.8ms	remaining: 8.89s
2:	learn: 936.1933927	total: 24.8ms	remaining: 8.24s
3:	learn: 878.1364825	total: 32.2ms	remaining: 8.02s
4:	learn: 826.5798346	total: 39ms	remaining: 7.76s
5:	learn: 781.0740671	total: 46.6ms	remaining: 7.72s
6:	learn: 739.6212641	total: 53ms	remaining: 7.52s
7:	learn: 703.3522461	total: 59.6ms	remaining: 7.39s
8:	learn: 670.7679130	total: 66.3ms	remaining: 7.3s
9:	learn: 641.7202792	total: 72.7ms	remaining: 7.2s
10:	learn: 616.5761763	total: 79.9ms	remaining: 7.19s
11:	learn: 593.9828954	total: 86.7ms	remaining: 7.14s
12:	learn: 574.1000032	total: 93.6ms	remaining: 7.1s
13:	learn: 556.8663475	total: 100ms	remaining: 7.06s
14:	learn: 541.5821335	total: 107ms	remaining: 6.99s
15:	learn: 528.3770580	total: 113ms	remaining: 6.95s
16:	learn: 516.8672895	total: 119ms	remaining: 6.87s
17:	learn: 506.2904876	total: 125ms	remaining: 6.82s
18:	learn: 497.558

## Preds

In [8]:
pd.set_option('display.max_columns', None)
X_B_test.head(3)

,absolute_humidity_2m:gm3,ceiling_height_agl:m,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,fresh_snow_1h:cm,fresh_snow_24h:cm,is_in_shadow:idx,msl_pressure:hPa,precip_type_5min:idx,pressure_50m:hPa,prob_rime:p,rain_water:kgm2,relative_humidity_1000hPa:p,sfc_pressure:hPa,snow_melt_10min:mm,snow_water:kgm2,sun_elevation:d,total_cloud_cover:p,visibility:m,wind_speed_10m:ms,wind_speed_w_1000hPa:ms,date_forecast,month,season,hours,sum_rad:W,rad_cloud_ratio
0,4.30,912.299988,1059.750000,0.0,271.650024,0.0,0.0,1.0,1013.775024,0.0,1006.674988,0.0,0.0,80.250000,1012.974976,0.0,0.0,-10.54025,75.000000,31107.599609,3.950,0.0,2023-05-01 00:00:00,0.522008,0.809017,0,0.0,0.0
1,4.25,1875.195190,1073.699951,0.0,271.450012,0.0,0.0,1.0,1013.250000,0.0,1006.174988,0.0,0.0,79.800003,1012.474976,0.0,0.0,-7.89400,77.500000,30409.699219,3.825,0.0,2023-05-01 01:00:00,0.522008,0.809017,1,0.0,0.0
2,4.15,1765.900024,1200.099976,0.0,271.049988,0.0,0.0,1.0,1012.700012,0.0,1005.650024,0.0,0.0,78.000000,1011.950012,0.0,0.0,-3.81500,88.099998,31342.650391,3.650,0.0,2023-05-01 02:00:00,0.522008,0.809017,2,0.0,0.0


In [14]:
pred_A = catModel_a.predict(X_A_test)
pred_B = catModel_bc.predict(X_B_test)
pred_C = catModel_bc.predict(X_C_test)

In [15]:
df_A = pd.DataFrame()

df_A["prediction"] = pred_A
df_A["location"] = "A"

df_B = pd.DataFrame()

df_B["prediction"] = pred_B
df_B["location"] = "B"

df_C = pd.DataFrame()

df_C["prediction"] = pred_C
df_C["location"] = "C"

df_mid = pd.concat([df_A, df_B], ignore_index=True)

df = pd.concat([df_mid, df_C], join="inner", ignore_index=True)

df = df.drop("location", axis=1)

# NAME THE FILE 
df.to_csv("sub32.csv")

df.shape

(2160, 1)